<div style="background-color:#B2EBF2; color:black;  font-size:10pt; border: 1px solid #35BDD0; border-radius: 5px; padding:10px" /div>
 <b>ЖИВОПЫРКА

In [2]:
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
# import io
# import msoffcrypto
from pathlib import Path
#from tqdm import tqdm
from IPython.display import display_markdown
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
file_облако = pd.read_excel('20-01-2025_16-00-00_Реестр_ЦПП.xlsx')
# file_облако_korr = pd.read_excel('28-10-2024_16-00_Реестр_ЦПП.xlsx')

In [4]:
file_емиас = pd.read_excel('Протокол ЕМИАС все визиты_20250123.xlsx')

In [251]:
df_obl = file_облако.copy()
df_емиас = file_емиас.copy()

In [252]:
# --------------------- Оцифруем ОМС_пациента_str и ID_пациента_str ----------------------------------------------------- преобразуем тип INT к СТРОКЕ 
df_емиас['ОМС_пациента_str'] = df_емиас['ОМС пациента'].apply(lambda x: str(int(x)) if not pd.isna(x) else str(x))
df_емиас['ID_пациента_str'] = df_емиас['ID пациента'].apply(lambda x: str(int(x)))

In [253]:
# ----------------------------- ключ - ID + дата плана ----------
df_емиас['id_data_fact'] = df_емиас['ID_пациента_str'] + '-' + df_емиас['Дата протокола'].astype('str')
df_емиас['id_data_plan'] = df_емиас['ID_пациента_str'] + '-' + df_емиас['план_дата_визит_врач'].astype('str')


In [254]:
# ------------------------------ ДУБЛИ в ЕМИАС!!!! -------------------------------------------------- убраны !! -----
df_емиас_uniq = df_емиас.drop_duplicates('id_data_fact')

In [255]:
# -------------------- ДУБЛИ в ЕМИАС!!!!!!!!!!!!!!!!!!!!!---------------------------------------------проверка 
# df_емиас[df_емиас.duplicated('id_data_fact', keep=False)].sort_values('план_дата_визит_врач')
df_емиас_uniq[df_емиас_uniq.duplicated('id_data_fact', keep=False)].sort_values('план_дата_визит_врач')

,ID пациента,ОМС пациента,Дата протокола,ФИО врача,Бригада врача,ОВПП,Тип протокола,Диагноз пациента,Показания к паллиативной помощи,Категория пациента,Оказание паллиативной помощи,Состояние пациента,Болевой синдром,Оценка болевого синдрома на момент осмотра в баллах (ВАШ),Оценка болевого синдрома на момент осмотра в баллах (НОШ),Оценка болевого синдрома на момент осмотра в баллах (painaid),Трахеостома,Респираторная поддержка,Мочеиспускание,Асцит,Установлено устройство,Дисфагия,Дисфагия. Степень,PPI,PPS,По шкале Морсе в баллах,По шкале Нортон в баллах,Описание помощи в амбулаторных условиях,Патронажный план,Частота посещений,Дата следующего планового визита,Дата следующего планового визита. С целью,Дата следующего планового визита. Причина,Манипуляции/Процедуры. Проведено,план_дата_визит_врач,ОМС_пациента_str,ID_пациента_str,id_data_fact,id_data_plan


In [256]:
# --------------------------------------- лишние колонки откинем
df_емиас_uniq = df_емиас_uniq[[
     'ОМС_пациента_str',
 'ID_пациента_str',
# 'ID пациента',
#  'ОМС пациента',
 'Дата протокола',
 'ФИО врача',
#  'Бригада врача',
 'ОВПП',
#  'Тип протокола',
#  'Манипуляции/Процедуры. Проведено',
 'план_дата_визит_врач',
]]

In [257]:
# df_obl.columns.tolist()

In [258]:
# ------------------------------------df_obl----- Оцифруем ОМС_пациента_str и ID_пациента_str --------------------------------------- преобразуем тип INT к СТРОКЕ 
df_obl['ОМС_пациента_str'] = df_obl['Полис ОМС'].apply(lambda x: str(x).replace('-','').replace(' ', '') )
df_obl['ID_пациента_str']  = df_obl['ID пациента'].apply(lambda x: str(x).replace('-','').replace(' ', '') )

In [259]:
# df_obl[df_obl['ID пациента'] == 18356054]

In [260]:
# --------------------------------------- лишние колонки откинем ----df_obl
df_obl = df_obl[[
     'ОМС_пациента_str',
 'ID_пациента_str',
# 'Дата включения пациента в Реестр',
 'ФИО пациента',
#  'ID пациента',
#  'Пол',
 'Дата рождения',
 'Гражданский статус',
#  'Адрес регистрации по месту жительства',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях',
#  'Основной диагноз',
 'Текущий статус пациента',
]]

In [261]:
df_емиас.shape

(59678, 39)

In [262]:
df_емиас_uniq.shape

(59459, 6)

<div style="background-color:BLUE; color:black;  font-size:10pt; border: 1px solid #35BDD0; border-radius: 5px; padding:10px" /div>
 <b> ВЫЧИСЛЯЕМ ОШИБКИ !!!!!

<div style="background-color:#B2EBF2; color:black;  font-size:10pt; border: 1px solid #35BDD0; border-radius: 5px; padding:10px" /div>
 <b>  ошибка 1 - НЕ НАЗНАЧЕН СЛЕДУЮЩИЙ

In [263]:
# --------------------- ошибка 1 - НЕ НАЗНАЧЕН СЛЕДУЮЩИЙ ------------------
df_емиас_uniq.loc[df_емиас_uniq['план_дата_визит_врач'].isna(), 'тип_дефекта'] = '1.не_указана_пл_дата'

In [264]:
# --------------------- ошибка 1 - НЕ НАЗНАЧЕН СЛЕДУЮЩИЙ ------------------
# df_емиас[df_емиас['план_дата_визит_врач'].isna()
#          & (df_емиас['Дата протокола'] >= '2025-01-10')
#          ].groupby('Дата протокола').agg({'Дата протокола':'count'})

In [265]:
# -------------------- создадим колонку, содержащую даты факта и плана 
df_емиас_uniq['f_p'] = df_емиас_uniq['Дата протокола'].astype(str) + '#' + df_емиас_uniq['план_дата_визит_врач'].astype(str)

In [266]:
#  ------------------------- ОТСОРТИРУЕМ по возрастанию строки по датам ФАКТА -> что раньше - выше (для правильного порядка пар в списках "все_визиты_и_планы")
df_емиас_uniq = df_емиас_uniq.sort_values('Дата протокола')

In [267]:
# ------- Формируем колонку, в которой ВСЕ пары f_p списком для каждой строки
df_емиас_uniq['все_визиты_и_планы'] = df_емиас_uniq['ID_пациента_str'].map(df_емиас_uniq.groupby('ID_пациента_str')['f_p'].apply(list))

In [268]:
# df_емиас_uniq[df_емиас_uniq['ID_пациента_str'] == '18356054']

In [269]:
# ------------------------------------------- ОШИБКА 2 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# df_емиас_uniq.loc[df_емиас_uniq['ID_пациента_str'] == '18356054'].loc['все_визиты_и_планы']

In [270]:
# ------------------------------------------ текст списка дат 
df_емиас_uniq.loc[45974    , 'все_визиты_и_планы']

['2024-05-14#NaT',
 '2024-09-11#2024-10-11',
 '2024-10-29#2024-11-25',
 '2024-12-23#2025-01-12',
 '2025-01-12#2025-01-31']

In [271]:
df_емиас_uniq

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы
45975,7773940895002381,22602700,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT]
45970,7701000115720649,10401098,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT]
45971,7700003052610542,29527448,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT]
45974,7473150832000034,21306347,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1..."
45973,7701003069600332,20920383,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-04#2024-10-04, 2024-1..."
...,...,...,...,...,...,...,...,...,...
31398,7701048010130339,24811776,2025-01-22,Исмаилов Р. Н.,"Филиал ""Савеловский"" ММЦПП ДЗМ",2025-02-04,NaN,2025-01-22#2025-02-04,"[2025-01-17#2025-01-22, 2025-01-22#2025-02-04]"
31397,7700007118540231,24406796,2025-01-22,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-02,NaN,2025-01-22#2025-02-02,"[2025-01-09#2025-01-14, 2025-01-14#2025-01-22,..."
31396,7702001090090748,16398757,2025-01-22,Колыхалкин А. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2025-02-09,NaN,2025-01-22#2025-02-09,[2025-01-22#2025-02-09]
31402,7700004089731140,10108639,2025-01-22,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2025-01-30,NaN,2025-01-22#2025-01-30,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1..."


In [272]:
df_емиас_uniq.loc[45974]

ОМС_пациента_str                                         7473150832000034
ID_пациента_str                                                  21306347
Дата протокола                                        2024-05-14 00:00:00
ФИО врача                                                  Беланова П. П.
ОВПП                                       Филиал "Коломенское" ММЦПП ДЗМ
план_дата_визит_врач                                                  NaT
тип_дефекта                                          1.не_указана_пл_дата
f_p                                                        2024-05-14#NaT
все_визиты_и_планы      [2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1...
Name: 45974, dtype: object

In [273]:
df_емиас_uniq.loc[df_емиас_uniq['ID_пациента_str'] == '21306347']

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы
45974,7473150832000034,21306347,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1..."
55710,7473150832000034,21306347,2024-09-11,Халиков Р. А.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2024-10-11,NaN,2024-09-11#2024-10-11,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1..."
7732,7473150832000034,21306347,2024-10-29,Халиков Р. А.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2024-11-25,NaN,2024-10-29#2024-11-25,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1..."
23117,7473150832000034,21306347,2024-12-23,Халиков Р. А.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-01-12,NaN,2024-12-23#2025-01-12,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1..."
28504,7473150832000034,21306347,2025-01-12,Халиков Р. А.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-01-31,NaN,2025-01-12#2025-01-31,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1..."


In [274]:
list_test = df_емиас_uniq.loc[55710, 'все_визиты_и_планы']
list_test

['2024-05-14#NaT',
 '2024-09-11#2024-10-11',
 '2024-10-29#2024-11-25',
 '2024-12-23#2025-01-12',
 '2025-01-12#2025-01-31']

In [275]:
list_test

['2024-05-14#NaT',
 '2024-09-11#2024-10-11',
 '2024-10-29#2024-11-25',
 '2024-12-23#2025-01-12',
 '2025-01-12#2025-01-31']

In [277]:
# error_list = []
# for i in list_test:
#     if i.split('#')[1] == 'NaT':   
#         # -------------- полполняем список ошибок - если нет плановой даты
#         error_list.append(('2024-05-14', list_test.index(i), '1.нет плана'))    
#     else:
#     # -------------- полполняем список ошибок - если не совпадает запланированная со следующим фактом
#         if i.split('#')[1] != (i+1).split('#')[1] :   
#             print(i.split('#')[0])
#             print(i.split('#')[1])

# print(error_list)



In [278]:
def insert_errors_tuple(list_test):
    '''Функция для записи кортежа ошибок в дфтафрейм'''
    error_list = []
    i = 0
    while i < len(list_test) - 1:
        if list_test[i].split('#')[1] == 'NaT':
            # -------------- полполняем список ошибок - если нет плановой даты
            error_list.append((list_test[i].split('#')[0], list_test[i].split('#')[1], '1.нет плана'))
        else:
        # -------------- полполняем список ошибок - если не совпадает запланированная со следующим фактом
            if list_test[i].split('#')[1] != list_test[i + 1].split('#')[0]:
                error_list.append((list_test[i + 1].split('#')[0], list_test[i].split('#')[1], '2.не совпал факт план'))
        i += 1
    # print(error_list)
    tuple_for_insert = error_list
    return tuple_for_insert

In [279]:
def insert_errors_list_dict(list_test):
    '''Функция для записи словаря  ошибок в дфтафрейм'''
    error_list_dict = [] 
    
    i = 0
    while i < len(list_test) - 1:
        error_dict = {}
        if list_test[i].split('#')[1] == 'NaT':
            # -------------- полполняем список ошибок - если нет плановой даты
            # error_list.append((list_test[i].split('#')[0], list_test[i].split('#')[1], '1.нет плана'))
            error_dict.update({list_test[i].split('#')[0] : [list_test[i].split('#')[1], '1.нет плана']})
        else:
        # -------------- полполняем список ошибок - если не совпадает запланированная со следующим фактом
            if list_test[i].split('#')[1] != list_test[i + 1].split('#')[0]:
                # error_list.append((list_test[i + 1].split('#')[0], list_test[i].split('#')[1], '2.не совпал факт план'))
                error_dict.update({list_test[i].split('#')[0] : [list_test[i].split('#')[1], '2.не совпал факт план']})
        error_list_dict.append(error_dict)
        i += 1
    # list_dict_for_insert = error_list_dict
    return error_list_dict

In [280]:
len(list_test)

5

In [281]:
list_test

['2024-05-14#NaT',
 '2024-09-11#2024-10-11',
 '2024-10-29#2024-11-25',
 '2024-12-23#2025-01-12',
 '2025-01-12#2025-01-31']

In [282]:
# ------------------- пробуем вставить в df -------------


In [283]:
# df_obl['ID пациента'].apply(lambda x: str(x).replace('-','').replace(' ', '') )
df_емиас_uniq['err_tup'] = df_емиас_uniq['все_визиты_и_планы'].apply(lambda x: insert_errors_tuple(x) if len(x) > 1 else None)
df_емиас_uniq['err_dict'] = df_емиас_uniq['все_визиты_и_планы'].apply(lambda x: insert_errors_list_dict(x) if len(x) > 1 else None)

In [284]:
df_емиас_uniq

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы,err_tup,err_dict
45975,7773940895002381,22602700,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None
45970,7701000115720649,10401098,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None
45971,7700003052610542,29527448,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None
45974,7473150832000034,21306347,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1...","[(2024-05-14, NaT, 1.нет плана), (2024-10-29, ...","[{'2024-05-14': ['NaT', '1.нет плана']}, {'202..."
45973,7701003069600332,20920383,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-04#2024-10-04, 2024-1...","[(2024-05-14, NaT, 1.нет плана)]","[{'2024-05-14': ['NaT', '1.нет плана']}, {}, {..."
...,...,...,...,...,...,...,...,...,...,...,...
31398,7701048010130339,24811776,2025-01-22,Исмаилов Р. Н.,"Филиал ""Савеловский"" ММЦПП ДЗМ",2025-02-04,NaN,2025-01-22#2025-02-04,"[2025-01-17#2025-01-22, 2025-01-22#2025-02-04]",[],[{}]
31397,7700007118540231,24406796,2025-01-22,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-02,NaN,2025-01-22#2025-02-02,"[2025-01-09#2025-01-14, 2025-01-14#2025-01-22,...",[],"[{}, {}]"
31396,7702001090090748,16398757,2025-01-22,Колыхалкин А. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2025-02-09,NaN,2025-01-22#2025-02-09,[2025-01-22#2025-02-09],None,None
31402,7700004089731140,10108639,2025-01-22,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2025-01-30,NaN,2025-01-22#2025-01-30,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {..."


In [160]:
df_емиас_uniq.loc[31374, 'все_визиты_и_планы']

['2024-12-29#2025-01-03',
 '2025-01-03#2025-01-14',
 '2025-01-14#2025-01-22',
 '2025-01-22#2025-01-30']

In [138]:
df_емиас_uniq.loc[59665, 'err']

[('2024-06-22', 'NaT', '1.нет плана'),
 ('2024-07-26', 'NaT', '1.нет плана'),
 ('2024-09-26', 'NaT', '1.нет плана'),
 ('2024-12-14', '2024-12-18', '2.не совпал факт план')]

In [338]:
df_емиас_uniq[df_емиас_uniq['ID_пациента_str'] == '10108639']

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы,err_tup,err_dict,x3
31666,7700004089731140,10108639,2024-08-06,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-08-06#NaT,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...",None
8028,7700004089731140,10108639,2024-10-30,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2024-11-27,NaN,2024-10-30#2024-11-27,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...",None
15689,7700004089731140,10108639,2024-11-27,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2024-12-25,NaN,2024-11-27#2024-12-25,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...",None
23431,7700004089731140,10108639,2024-12-25,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2025-01-22,NaN,2024-12-25#2025-01-22,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...",None
31402,7700004089731140,10108639,2025-01-22,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2025-01-30,NaN,2025-01-22#2025-01-30,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...",None


In [285]:
df_емиас_uniq.loc[45974, 'err_dict']

[{'2024-05-14': ['NaT', '1.нет плана']},
 {'2024-09-11': ['2024-10-11', '2.не совпал факт план']},
 {'2024-10-29': ['2024-11-25', '2.не совпал факт план']},
 {}]

In [335]:
def print_person_error(list_dict, date_visit):
        
    key_ = str(date_visit).split(' ')[0]
    date_errors = '-'
    if list_dict is not None:
        for dict_ in list_dict:
            
            # if key_ in dict_.get(key_):
            if dict_.get(key_) is not None:
                date_errors = dict_.get(key_)
                break

    return date_errors

In [315]:
df_емиас_uniq['err_dict'].apply(lambda x: x)

45975                                                 None
45970                                                 None
45971                                                 None
45974    [{'2024-05-14': ['NaT', '1.нет плана']}, {'202...
45973    [{'2024-05-14': ['NaT', '1.нет плана']}, {}, {...
                               ...                        
31398                                                 [{}]
31397                                             [{}, {}]
31396                                                 None
31402    [{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...
31374                                         [{}, {}, {}]
Name: err_dict, Length: 59459, dtype: object

In [288]:
df_емиас_uniq.apply(lambda x: x[31402    ])

ОМС_пациента_str                                         7700004089731140
ID_пациента_str                                                  10108639
Дата протокола                                        2025-01-22 00:00:00
ФИО врача                                                    Хрущев А. К.
ОВПП                                        Филиал "Зеленоград" ММЦПП ДЗМ
план_дата_визит_врач                                  2025-01-30 00:00:00
тип_дефекта                                                           NaN
f_p                                                 2025-01-22#2025-01-30
все_визиты_и_планы      [2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...
err_tup                                  [(2024-08-06, NaT, 1.нет плана)]
err_dict                [{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...
dtype: object

In [321]:
df_емиас_uniq['Дата протокола'].astype(str)

45975    2024-05-14
45970    2024-05-14
45971    2024-05-14
45974    2024-05-14
45973    2024-05-14
            ...    
31398    2025-01-22
31397    2025-01-22
31396    2025-01-22
31402    2025-01-22
31374    2025-01-22
Name: Дата протокола, Length: 59459, dtype: object

In [337]:
df_емиас_uniq.loc[10108639    , 'Дата протокола']

KeyError: 10108639

In [316]:
df_емиас_uniq0['x3'] = df_емиас_uniq0.apply(lambda x: next((y for y in x['err_dict'] if y.get(x['Дата протокола']) is not None), None), axis=1)

In [339]:
df_емиас_uniq['ошибка_в_дату'] = df_емиас_uniq.apply(lambda x: print_person_error(x['err_dict'], x['Дата протокола']), axis=1)

In [340]:
df_емиас_uniq

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы,err_tup,err_dict,x3,ошибка_в_дату
45975,7773940895002381,22602700,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None,-
45970,7701000115720649,10401098,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None,-
45971,7700003052610542,29527448,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None,-
45974,7473150832000034,21306347,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1...","[(2024-05-14, NaT, 1.нет плана), (2024-10-29, ...","[{'2024-05-14': ['NaT', '1.нет плана']}, {'202...",None,"[NaT, 1.нет плана]"
45973,7701003069600332,20920383,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-04#2024-10-04, 2024-1...","[(2024-05-14, NaT, 1.нет плана)]","[{'2024-05-14': ['NaT', '1.нет плана']}, {}, {...",None,"[NaT, 1.нет плана]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31398,7701048010130339,24811776,2025-01-22,Исмаилов Р. Н.,"Филиал ""Савеловский"" ММЦПП ДЗМ",2025-02-04,NaN,2025-01-22#2025-02-04,"[2025-01-17#2025-01-22, 2025-01-22#2025-02-04]",[],[{}],None,-
31397,7700007118540231,24406796,2025-01-22,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-02,NaN,2025-01-22#2025-02-02,"[2025-01-09#2025-01-14, 2025-01-14#2025-01-22,...",[],"[{}, {}]",None,-
31396,7702001090090748,16398757,2025-01-22,Колыхалкин А. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2025-02-09,NaN,2025-01-22#2025-02-09,[2025-01-22#2025-02-09],None,None,None,-
31402,7700004089731140,10108639,2025-01-22,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2025-01-30,NaN,2025-01-22#2025-01-30,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...",None,-


In [352]:
df_емиас_uniq[(df_емиас_uniq['Дата протокола'] == '2024-11-17')
              & ((df_емиас_uniq['ошибка_в_дату'] != '-') | (df_емиас_uniq['ошибка_в_дату'] == '1.не_указана_пл_дата'))
              ]

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы,err_tup,err_dict,x3,ошибка_в_дату
13000,7700009121810164,25738060,2024-11-17,Паршиков В. В.,"Филиал ""Люблино"" ОРП ММЦПП ДЗМ",2024-12-17,NaN,2024-11-17#2024-12-17,"[2024-09-26#2024-10-09, 2024-10-28#2024-11-02,...","[(2024-10-28, 2024-10-09, 2.не совпал факт пла...","[{'2024-09-26': ['2024-10-09', '2.не совпал фа...",None,"[2024-12-17, 2.не совпал факт план]"
13018,7774950820000136,17261523,2024-11-17,Орлов Е. С.,"Филиал ""Бутово"" ММЦПП ДЗМ",2024-12-16,NaN,2024-11-17#2024-12-16,"[2024-08-18#2024-08-23, 2024-08-25#2024-09-07,...","[(2024-08-25, 2024-08-23, 2.не совпал факт пла...","[{'2024-08-18': ['2024-08-23', '2.не совпал фа...",None,"[2024-12-16, 2.не совпал факт план]"
13007,7752920888001850,21019778,2024-11-17,Эльдарова А. З.,"Филиал ""Савеловский"" ММЦПП ДЗМ",2024-11-22,NaN,2024-11-17#2024-11-22,"[2024-10-31#2024-11-05, 2024-11-05#2024-11-12,...","[(2024-12-10, 2024-11-22, 2.не совпал факт план)]","[{}, {}, {}, {'2024-11-17': ['2024-11-22', '2....",None,"[2024-11-22, 2.не совпал факт план]"
13010,5067260882000523,30560721,2024-11-17,Григорьева И. В.,"Филиал ""Вороново"" ММЦПП ДЗМ",2024-12-18,NaN,2024-11-17#2024-12-18,"[2024-06-11#NaT, 2024-08-10#NaT, 2024-09-21#20...","[(2024-06-11, NaT, 1.нет плана), (2024-08-10, ...","[{'2024-06-11': ['NaT', '1.нет плана']}, {'202...",None,"[2024-12-18, 2.не совпал факт план]"
13011,7753640891000421,16213912,2024-11-17,Элмуратов Б. Ш.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2024-12-15,NaN,2024-11-17#2024-12-15,"[2024-07-18#NaT, 2024-09-19#2024-10-19, 2024-1...","[(2024-07-18, NaT, 1.нет плана), (2024-12-31, ...","[{'2024-07-18': ['NaT', '1.нет плана']}, {}, {...",None,"[2024-12-15, 2.не совпал факт план]"
13020,7754740877001424,10167790,2024-11-17,Гладкова В. В.,"Филиал ""Ростокино"" ММЦПП ДЗМ",2024-12-17,NaN,2024-11-17#2024-12-17,"[2024-10-27#2024-11-03, 2024-11-03#2024-11-17,...","[(2024-12-22, 2024-12-17, 2.не совпал факт план)]","[{}, {}, {'2024-11-17': ['2024-12-17', '2.не с...",None,"[2024-12-17, 2.не совпал факт план]"
12971,7754440881002142,18518751,2024-11-17,Коновод И. А.,"Филиал ""Преображенское"" ММЦПП ДЗМ",2024-11-22,NaN,2024-11-17#2024-11-22,"[2024-11-17#2024-11-22, 2024-11-18#2024-11-22,...","[(2024-11-18, 2024-11-22, 2.не совпал факт план)]","[{'2024-11-17': ['2024-11-22', '2.не совпал фа...",None,"[2024-11-22, 2.не совпал факт план]"
12978,7769850874000296,23495121,2024-11-17,Бозгулова С. П.,"Филиал ""Хорошёво"" ММЦПП ДЗМ",2024-11-23,NaN,2024-11-17#2024-11-23,"[2024-10-18#2024-11-01, 2024-11-01#2024-11-10,...","[(2024-11-25, 2024-11-23, 2.не совпал факт план)]","[{}, {}, {}, {'2024-11-17': ['2024-11-23', '2....",None,"[2024-11-23, 2.не совпал факт план]"
12981,7700005144670342,10213841,2024-11-17,Юлдашев И. И.,"Филиал ""Царицыно"" ММЦПП ДЗМ",2024-11-23,NaN,2024-11-17#2024-11-23,"[2024-11-17#2024-11-23, 2024-12-07#2024-12-21]","[(2024-12-07, 2024-11-23, 2.не совпал факт план)]","[{'2024-11-17': ['2024-11-23', '2.не совпал фа...",None,"[2024-11-23, 2.не совпал факт план]"
12985,7700001141720340,10270884,2024-11-17,Филюшкина И. Ю.,"Филиал ""Царицыно"" ММЦПП ДЗМ",2024-11-29,NaN,2024-11-17#2024-11-29,"[2024-11-10#2024-11-17, 2024-11-17#2024-11-29,...","[(2024-12-27, 2024-11-29, 2.не совпал факт план)]","[{}, {'2024-11-17': ['2024-11-29', '2.не совпа...",None,"[2024-11-29, 2.не совпал факт план]"


In [302]:
import pandas as pd

# создаем датафрейм
df_емиас_uniq1 = pd.DataFrame({
    'Дата протокола': ['2022-01-01', '2022-01-02', '2022-01-04'],
    'err_dict': [
        [{'2022-01-01': 'error1'}, {'2022-01-02': 'error2'}],
        [{'2022-01-04': 'error3'}, {'2022-01-03': 'error4'}],
        [{'2022-01-03': 'error5'}, {'2022-01-04': 'error6'}]
    ]
})

# создаем новую колонку x3
df_емиас_uniq1['x3'] = df_емиас_uniq1.apply(lambda x: next((y for y in x['err_dict'] if y.get(x['Дата протокола']) is not None), None), axis=1)

df_емиас_uniq1

,Дата протокола,err_dict,x3
0,2022-01-01,"[{'2022-01-01': 'error1'}, {'2022-01-02': 'err...",{'2022-01-01': 'error1'}
1,2022-01-02,"[{'2022-01-04': 'error3'}, {'2022-01-03': 'err...",None
2,2022-01-04,"[{'2022-01-03': 'error5'}, {'2022-01-04': 'err...",{'2022-01-04': 'error6'}


In [306]:
df_емиас_uniq0 = df_емиас_uniq[~df_емиас_uniq['err_dict'].isna()]

In [294]:
df_емиас_uniq.apply(lambda x: print_person_error(x['err_dict'], x['Дата протокола']), axis=1)

UnboundLocalError: local variable 'date_errors' referenced before assignment

In [292]:
df_емиас_uniq

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы,err_tup,err_dict
45975,7773940895002381,22602700,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None
45970,7701000115720649,10401098,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None
45971,7700003052610542,29527448,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None
45974,7473150832000034,21306347,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1...","[(2024-05-14, NaT, 1.нет плана), (2024-10-29, ...","[{'2024-05-14': ['NaT', '1.нет плана']}, {'202..."
45973,7701003069600332,20920383,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-04#2024-10-04, 2024-1...","[(2024-05-14, NaT, 1.нет плана)]","[{'2024-05-14': ['NaT', '1.нет плана']}, {}, {..."
...,...,...,...,...,...,...,...,...,...,...,...
31398,7701048010130339,24811776,2025-01-22,Исмаилов Р. Н.,"Филиал ""Савеловский"" ММЦПП ДЗМ",2025-02-04,NaN,2025-01-22#2025-02-04,"[2025-01-17#2025-01-22, 2025-01-22#2025-02-04]",[],[{}]
31397,7700007118540231,24406796,2025-01-22,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-02,NaN,2025-01-22#2025-02-02,"[2025-01-09#2025-01-14, 2025-01-14#2025-01-22,...",[],"[{}, {}]"
31396,7702001090090748,16398757,2025-01-22,Колыхалкин А. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2025-02-09,NaN,2025-01-22#2025-02-09,[2025-01-22#2025-02-09],None,None
31402,7700004089731140,10108639,2025-01-22,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2025-01-30,NaN,2025-01-22#2025-01-30,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {..."


In [ ]:
import pandas as pd

# создаем датафрейм
df_емиас_uniq = pd.DataFrame({
    'колонка1': [1, 2, 3, 4, 5],
    'колонка2': [10, 20, 30, 40, 50]
})

# создаем функцию
def def_res(x1, x2):
    return x1 + x2

# применяем функцию к датафрейму
df_емиас_uniq.apply(lambda x: def_res(x['колонка1'], x['колонка2']), axis=1)

# print(df_емиас_uniq)

0    11
1    22
2    33
3    44
4    55
dtype: int64

In [ ]:
df_емиас_uniq['err_dict'].apply(lambda x: print_person_error(df_емиас_uniq['err_dict'].apply(lambda x: x),df_емиас_uniq['Дата протокола'].apply(lambda x: x)))

45975                                      9
45970                                      9
45971                                      9
45974    [2024-10-11, 2.не совпал факт план]
45973                                      9
                        ...                 
31398                                      9
31397                                      9
31396                                      9
31402                                      9
31374                                      9
Name: err_dict, Length: 59459, dtype: object

In [ ]:
for i in 
df_емиас_uniq.apply(lambda x: print_person_error(x['err_dict'], x['Дата протокола']), axis=1)


45975    9
45970    9
45971    9
45974    9
45973    9
        ..
31398    9
31397    9
31396    9
31402    9
31374    9
Length: 59459, dtype: object

In [220]:
df_емиас_uniq.apply(print_person_error(x['err_dict'], x['Дата протокола']))

NameError: name 'x' is not defined

In [202]:
df_емиас_uniq[df_емиас_uniq['err_dict'].get(df_емиас_uniq['Дата протокола']) is None, 'www'  ] = df_емиас_uniq['err_dict'].get(df_емиас_uniq['Дата протокола'])
    
    # df_емиас_uniq['Дата протокола'] ]

In [203]:
df_емиас_uniq

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы,err,err_tup,err_dict,"(False, www)","(True, www)"
45975,7773940895002381,22602700,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None,None,None
45970,7701000115720649,10401098,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None,None,None
45971,7700003052610542,29527448,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None,None,None
45974,7473150832000034,21306347,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1...","[(2024-05-14, NaT, 1.нет плана), (2024-10-29, ...","[(2024-05-14, NaT, 1.нет плана), (2024-10-29, ...","[{'2024-05-14': ['NaT', '1.нет плана']}, {'202...",None,None
45973,7701003069600332,20920383,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-04#2024-10-04, 2024-1...","[(2024-05-14, NaT, 1.нет плана)]","[(2024-05-14, NaT, 1.нет плана)]","[{'2024-05-14': ['NaT', '1.нет плана']}, {}, {...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31398,7701048010130339,24811776,2025-01-22,Исмаилов Р. Н.,"Филиал ""Савеловский"" ММЦПП ДЗМ",2025-02-04,NaN,2025-01-22#2025-02-04,"[2025-01-17#2025-01-22, 2025-01-22#2025-02-04]",[],[],[{}],None,None
31397,7700007118540231,24406796,2025-01-22,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-02,NaN,2025-01-22#2025-02-02,"[2025-01-09#2025-01-14, 2025-01-14#2025-01-22,...",[],[],"[{}, {}]",None,None
31396,7702001090090748,16398757,2025-01-22,Колыхалкин А. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2025-02-09,NaN,2025-01-22#2025-02-09,[2025-01-22#2025-02-09],None,None,None,None,None
31402,7700004089731140,10108639,2025-01-22,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2025-01-30,NaN,2025-01-22#2025-01-30,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...",None,None


In [201]:
df_емиас_uniq['err_dict'].get('2024-10-05')

In [205]:
df_емиас_uniq['err_dict']
# .get(df_емиас_uniq['Дата протокола'])

45975                                                 None
45970                                                 None
45971                                                 None
45974    [{'2024-05-14': ['NaT', '1.нет плана']}, {'202...
45973    [{'2024-05-14': ['NaT', '1.нет плана']}, {}, {...
                               ...                        
31398                                                 [{}]
31397                                             [{}, {}]
31396                                                 None
31402    [{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...
31374                                         [{}, {}, {}]
Name: err_dict, Length: 59459, dtype: object

In [197]:
df_емиас_uniq[df_емиас_uniq['err_dict'].get(df_емиас_uniq['Дата протокола'])]

KeyError: None

In [192]:
df_емиас_uniq

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы,err,err_tup,err_dict,"(False, www)"
45975,7773940895002381,22602700,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None,None
45970,7701000115720649,10401098,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None,None
45971,7700003052610542,29527448,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None,None
45974,7473150832000034,21306347,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1...","[(2024-05-14, NaT, 1.нет плана), (2024-10-29, ...","[(2024-05-14, NaT, 1.нет плана), (2024-10-29, ...","[{'2024-05-14': ['NaT', '1.нет плана']}, {'202...",None
45973,7701003069600332,20920383,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-04#2024-10-04, 2024-1...","[(2024-05-14, NaT, 1.нет плана)]","[(2024-05-14, NaT, 1.нет плана)]","[{'2024-05-14': ['NaT', '1.нет плана']}, {}, {...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31398,7701048010130339,24811776,2025-01-22,Исмаилов Р. Н.,"Филиал ""Савеловский"" ММЦПП ДЗМ",2025-02-04,NaN,2025-01-22#2025-02-04,"[2025-01-17#2025-01-22, 2025-01-22#2025-02-04]",[],[],[{}],None
31397,7700007118540231,24406796,2025-01-22,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-02,NaN,2025-01-22#2025-02-02,"[2025-01-09#2025-01-14, 2025-01-14#2025-01-22,...",[],[],"[{}, {}]",None
31396,7702001090090748,16398757,2025-01-22,Колыхалкин А. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2025-02-09,NaN,2025-01-22#2025-02-09,[2025-01-22#2025-02-09],None,None,None,None
31402,7700004089731140,10108639,2025-01-22,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2025-01-30,NaN,2025-01-22#2025-01-30,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {...",None


In [188]:
# df_емиас_uniq['err_dict'] = 
df_емиас_uniq.apply(lambda x: print_person_error(x['Дата протокола', 'err_dict']))

KeyError: 'key of type tuple not found and not a MultiIndex'

In [184]:
df_емиас_uniq

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы,err,err_tup,err_dict
45975,7773940895002381,22602700,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None
45970,7701000115720649,10401098,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None
45971,7700003052610542,29527448,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT],None,None,None
45974,7473150832000034,21306347,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-11#2024-10-11, 2024-1...","[(2024-05-14, NaT, 1.нет плана), (2024-10-29, ...","[(2024-05-14, NaT, 1.нет плана), (2024-10-29, ...","[{'2024-05-14': ['NaT', '1.нет плана']}, {'202..."
45973,7701003069600332,20920383,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-05-14#NaT, 2024-09-04#2024-10-04, 2024-1...","[(2024-05-14, NaT, 1.нет плана)]","[(2024-05-14, NaT, 1.нет плана)]","[{'2024-05-14': ['NaT', '1.нет плана']}, {}, {..."
...,...,...,...,...,...,...,...,...,...,...,...,...
31398,7701048010130339,24811776,2025-01-22,Исмаилов Р. Н.,"Филиал ""Савеловский"" ММЦПП ДЗМ",2025-02-04,NaN,2025-01-22#2025-02-04,"[2025-01-17#2025-01-22, 2025-01-22#2025-02-04]",[],[],[{}]
31397,7700007118540231,24406796,2025-01-22,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-02,NaN,2025-01-22#2025-02-02,"[2025-01-09#2025-01-14, 2025-01-14#2025-01-22,...",[],[],"[{}, {}]"
31396,7702001090090748,16398757,2025-01-22,Колыхалкин А. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2025-02-09,NaN,2025-01-22#2025-02-09,[2025-01-22#2025-02-09],None,None,None
31402,7700004089731140,10108639,2025-01-22,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2025-01-30,NaN,2025-01-22#2025-01-30,"[2024-08-06#NaT, 2024-10-30#2024-11-27, 2024-1...","[(2024-08-06, NaT, 1.нет плана)]","[(2024-08-06, NaT, 1.нет плана)]","[{'2024-08-06': ['NaT', '1.нет плана']}, {}, {..."


In [144]:
df_емиас_uniq['err'].str[0]

45975                               NaN
45970                               NaN
45971                               NaN
45974    (2024-05-14, NaT, 1.нет плана)
45973    (2024-05-14, NaT, 1.нет плана)
                      ...              
31398                               NaN
31397                               NaN
31396                               NaN
31402    (2024-08-06, NaT, 1.нет плана)
31374                               NaN
Name: err, Length: 59459, dtype: object

In [146]:
df_емиас_uniq['err'].str[1][0]

('2024-07-31', 'NaT', '1.нет плана')

In [143]:
df_емиас_uniq['err']

45975                                                   []
45970                                                   []
45971                                                   []
45974    [(2024-05-14, NaT, 1.нет плана), (2024-10-29, ...
45973                     [(2024-05-14, NaT, 1.нет плана)]
                               ...                        
31398                                                   []
31397                                                   []
31396                                                   []
31402                     [(2024-08-06, NaT, 1.нет плана)]
31374                                                   []
Name: err, Length: 59459, dtype: object

In [139]:
df_емиас_uniq.loc[59665]

ОМС_пациента_str                                         7700001103651151
ID_пациента_str                                                  18356054
Дата протокола                                        2025-01-12 00:00:00
ФИО врача                                                Гладенкова А. О.
ОВПП                                          Филиал "Солнцево" ММЦПП ДЗМ
план_дата_визит_врач                                  2025-02-08 00:00:00
тип_дефекта                                                           NaN
f_p                                                 2025-01-12#2025-02-08
все_визиты_и_планы      [2024-06-22#NaT, 2024-07-26#NaT, 2024-08-25#20...
err                     [(2024-06-22, NaT, 1.нет плана), (2024-07-26, ...
Name: 59665, dtype: object

In [41]:
#  ------------------------- ОТСОРТИРУЕМ по возрастанию строки по датам ФАКТА -> что раньше - выше (для правильного порядка пар в списках "все_визиты_и_планы")
df_емиас_uniq.sort_values('Дата протокола')

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,все_визиты_и_планы
45975,77739 89 02381,22 2 0,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT]
45970,7 1157 649,98,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT]
45971,7 0 526 542,29527448,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,[2024-05-14#NaT]
45974,74731 83 034,21 6347,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-10-29#2024-11-25, 2024-12-23#2025-01-12,..."
45973,7 0 69 0332,9 383,2024-05-14,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-05-14#NaT,"[2024-11-03#2024-11-24, 2024-11-24#2024-12-23,..."
...,...,...,...,...,...,...,...,...,...
31398,7 4 1 339,24811776,2025-01-22,Исмаилов Р. Н.,"Филиал ""Савеловский"" ММЦПП ДЗМ",2025-02-04,NaN,2025-01-22#2025-02-04,"[2025-01-17#2025-01-22, 2025-01-22#2025-02-04]"
31397,7 071185 231,24 6796,2025-01-22,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-02,NaN,2025-01-22#2025-02-02,"[2025-01-09#2025-01-14, 2025-01-14#2025-01-22,..."
31396,7 0 0 748,16398757,2025-01-22,Колыхалкин А. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2025-02-09,NaN,2025-01-22#2025-02-09,[2025-01-22#2025-02-09]
31402,7 0 897311,8639,2025-01-22,Хрущев А. К.,"Филиал ""Зеленоград"" ММЦПП ДЗМ",2025-01-30,NaN,2025-01-22#2025-01-30,"[2024-10-30#2024-11-27, 2024-11-27#2024-12-25,..."


In [71]:
df_емиас_uniq[~df_емиас_uniq['ID_пациента_str'].isna()
              & df_емиас_uniq['ID_пациента_str'].str.contains(' ')]

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p
0,777 84 526,18 2538,2024-09-30,Моисеева Т. В.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-09-30#NaT
3,77 2 82 0 47,2632 53,2024-09-30,Лямзин С. А.,"Филиал ""Люблино"" ОРП ММЦПП ДЗМ",2024-09-30,NaN,2024-09-30#2024-09-30
5,7 061 2 852,152756,2024-09-30,Казак А. А.,"Филиал ""Даниловский"" ММЦПП ДЗМ",2024-10-05,NaN,2024-09-30#2024-10-05
6,7 0414 677,1668 78,2024-09-30,Лямзин С. А.,"Филиал ""Люблино"" ОРП ММЦПП ДЗМ",2024-10-30,NaN,2024-09-30#2024-10-30
7,77561 87 312,277928,2024-09-30,Казак А. А.,"Филиал ""Даниловский"" ММЦПП ДЗМ",2024-10-05,NaN,2024-09-30#2024-10-05
...,...,...,...,...,...,...,...,...
59660,77681 84 048,695765,2025-01-12,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",2025-02-09,NaN,2025-01-12#2025-02-09
59661,77671 87 01827,244 583,2025-01-12,Васильева Е. П.,"Филиал ""Солнцево"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2025-01-12#NaT
59663,9 89 687,1674 06,2025-01-12,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",2025-02-08,NaN,2025-01-12#2025-02-08
59664,7 0 015 354,4111266,2025-01-12,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",2025-02-08,NaN,2025-01-12#2025-02-08


In [74]:
df_емиас_uniq[df_емиас_uniq['ID_пациента_str'] == '1835 54']

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p
20867,7 01 3651151,1835 54,2024-12-14,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",2025-01-12,NaN,2024-12-14#2025-01-12
34317,7 01 3651151,1835 54,2024-07-26,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-07-26#NaT
41705,7 01 3651151,1835 54,2024-06-22,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-06-22#NaT
46203,7 01 3651151,1835 54,2024-08-25,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",2024-09-26,NaN,2024-08-25#2024-09-26
49333,7 01 3651151,1835 54,2024-11-17,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",2024-12-18,NaN,2024-11-17#2024-12-18
51866,7 01 3651151,1835 54,2024-09-26,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-09-26#NaT
59665,7 01 3651151,1835 54,2025-01-12,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",2025-02-08,NaN,2025-01-12#2025-02-08


In [76]:
df_емиас_uniq[df_емиас_uniq['ID_пациента_str'] == 183554]

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p


In [ ]:
# for i in df_емиас['ID_пациента_str'].unique():
#     df_емиас[df_емиас['ID_пациента_str'] == i, 'f_p'] = df_емиас['f_p'] + 

In [32]:
# df_емиас['список_всех_пар_факт_план'] = df_емиас.groupby('ID_пациента_str')['f_p'].transform(lambda x: list(x))

In [33]:
# df['f2'] = df.groupby('ID_пациента_str')['f_p'].transform(lambda x: list(x))

In [34]:
# df_емиас.groupby('ID_пациента_str').agg({'f_p':'count'})

In [35]:
# df_емиас[df_емиас['ID_пациента_str'] == '1003289391']

In [36]:
# Пример DataFrame
# data = {
#     'ОМС_пациента_str': ['123', '456', '123', '789', '456'],
#     'ID_пациента_str': ['A', 'B', 'A', 'C', 'B'],
#     'Дата протокола': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'],
#     'ФИО врача': ['Иванов И.И.', 'Петров П.П.', 'Иванов И.И.', 'Сидоров С.С.', 'Петров П.П.'],
#     'ОВПП': ['Да', 'Нет', 'Да', 'Да', 'Нет'],
#     'план_дата_визит_врач': ['2023-01-10', '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-14'],
#     'f_p': [1, 2, 3, 4, 5]
# }

# df = pd.DataFrame(data)

In [37]:
# Преобразуем f2 в списки для каждой строки
# df['f2'] = df['ID_пациента_str'].map(df.groupby('ID_пациента_str')['f_p'].apply(list))
# df

In [471]:
df_емиас[df_емиас['ID_пациента_str'] == '1003289391']
# Преобразуем f2 в списки для каждой строки
df_емиас['f2'] = df_емиас['ID_пациента_str'].map(df_емиас.groupby('ID_пациента_str')['f_p'].apply(list))


In [480]:
df_емиас[df_емиас['ID_пациента_str'] == '1003289391'].sort_values('Дата протокола')


,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта,f_p,список_всех_пар_факт_план,f2
39825,7750130877001992,1003289391,2024-06-30,Разживина Е. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-06-30#NaT,2024-06-30#NaT,"[2024-09-30#2024-10-16, 2024-10-16#2024-11-15,..."
35501,7750130877001992,1003289391,2024-07-20,Разживина Е. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-07-20#NaT,2024-07-20#NaT,"[2024-09-30#2024-10-16, 2024-10-16#2024-11-15,..."
54304,7750130877001992,1003289391,2024-09-16,Колыхалкин А. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2024-10-16,NaN,2024-09-16#2024-10-16,2024-09-16#2024-10-16,"[2024-09-30#2024-10-16, 2024-10-16#2024-11-15,..."
14,7750130877001992,1003289391,2024-09-30,Самозван Ю. Н.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2024-10-16,NaN,2024-09-30#2024-10-16,2024-09-30#2024-10-16,"[2024-09-30#2024-10-16, 2024-10-16#2024-11-15,..."
4008,7750130877001992,1003289391,2024-10-16,Разживина Е. А.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2024-11-15,NaN,2024-10-16#2024-11-15,2024-10-16#2024-11-15,"[2024-09-30#2024-10-16, 2024-10-16#2024-11-15,..."
12375,7750130877001992,1003289391,2024-11-15,Егоров Е. В.,"Филиал ""Дегунино"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата,2024-11-15#NaT,2024-11-15#NaT,"[2024-09-30#2024-10-16, 2024-10-16#2024-11-15,..."


In [434]:
df_емиас.groupby('ID_пациента_str').agg({'Дата протокола':'count'})

,Дата протокола
ID_пациента_str,
1001786741,1
1001866691,2
1003289391,6
1003931041,2
1004736691,1
...,...
986580866,2
987357816,5
987625341,1


In [ ]:
# ---- А - визит до плановой даты -----------

In [433]:
df_емиас

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,тип_дефекта
0,7772060844000526,18902538,2024-09-30,Моисеева Т. В.,"Филиал ""Коломенское"" ММЦПП ДЗМ",NaT,1.не_указана_пл_дата
1,7701004055610961,19648959,2024-09-30,Борисова Т. П.,"Филиал ""Царицыно"" ММЦПП ДЗМ",2024-10-14,NaN
2,7774160878000109,18131229,2024-09-30,Разгуляев А. В.,"Филиал ""Куркино"" ММЦПП ДЗМ",2024-10-17,NaN
3,7770250825001047,26324053,2024-09-30,Лямзин С. А.,"Филиал ""Люблино"" ОРП ММЦПП ДЗМ",2024-09-30,NaN
4,7774360877000627,2242663271,2024-09-30,Ледяева О. В.,"Филиал ""Вороново"" ММЦПП ДЗМ",2024-10-31,NaN
...,...,...,...,...,...,...,...
59673,7700001164070648,20815226,2024-09-19,Егоров Е. В.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2024-10-03,NaN
59674,7774460895000219,10158361,2024-12-17,Уварова О. В.,"Филиал ""Преображенское"" ММЦПП ДЗМ",2025-01-16,NaN
59675,205494272,21958494,2024-09-12,Самойлова С. В.,"Филиал ""Куркино"" ММЦПП ДЗМ",2024-10-10,NaN
59676,7774460895000219,10158361,2024-09-08,Уварова О. В.,"Филиал ""Преображенское"" ММЦПП ДЗМ",2024-09-22,NaN


In [431]:
df_емиас.loc[df_емиас['план_дата_визит_врач'].isna(), 'тип_дефекта'] = '1.не_указана_пл_дата'

In [432]:
df_емиас['ID_пациента_str'].nunique()

17023

In [ ]:
# --------------------- 2...3 Невыполнение плана  ------------------


In [ ]:
# ---- А - визит до плановой даты -----------

In [ ]:
# ---- B - визит после плановой даты -----------

In [428]:
df_емиас[df_емиас['план_дата_визит_врач'].isna()
         & (df_емиас['Дата протокола'] == '2025-01-10')
         ]

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач
27881,7756040875002633,22697093,2025-01-10,Ревека Ю. И.,"Филиал ""Дегунино"" ММЦПП ДЗМ",NaT
27898,7769550876000910,30212910,2025-01-10,Фрибус Е. С.,"Филиал ""Царицыно"" ММЦПП ДЗМ",NaT
27904,7700006119760263,24885141,2025-01-10,Дундуков Д. А.,"Филиал ""Некрасовка"" ММЦПП ДЗМ",NaT
27921,7748530844000214,31071041,2025-01-10,Савельев В. Н.,"Филиал ""Царицыно"" ММЦПП ДЗМ",NaT
27944,7773350875001249,20633957,2025-01-10,Ревека Ю. И.,"Филиал ""Дегунино"" ММЦПП ДЗМ",NaT
27947,7758930894000295,20052385,2025-01-10,Абдуллаева Т. Д.,"Филиал ""Люблино"" ММЦПП ДЗМ",NaT
27959,7773950828000857,1031890991,2025-01-10,Спахов В. В.,"Филиал ""Дегунино"" ММЦПП ДЗМ",NaT
27987,7753730869003380,30000011308435,2025-01-10,Казарян С. С.,"Филиал ""Люблино"" ММЦПП ДЗМ",NaT
27995,7701001042780393,17436644,2025-01-10,Самойлова С. В.,"Филиал ""Куркино"" ММЦПП ДЗМ",NaT
28001,7700007075160264,10722450,2025-01-10,Олексюк М. С.,"Филиал ""Ростокино"" ММЦПП ДЗМ",NaT


In [357]:
#  -------------------------- убрали ПУСТЫЕ ПЛАНОВЫЕ ДАТЫ -----------------
df_емиас = df_емиас[~df_емиас['план_дата_визит_врач'].isna()]

In [358]:
# ----------------- ключ - ID + дата плана ----------
df_емиас['id_data_fact'] = df_емиас['ID_пациента_str'] + '-' + df_емиас['Дата протокола'].astype('str')
df_емиас['id_data_plan'] = df_емиас['ID_пациента_str'] + '-' + df_емиас['план_дата_визит_врач'].astype('str')

<div style="background-color:green; color:black;  font-size:10pt; border: 1px solid #35BDD0; border-radius: 5px; padding:10px" /div>
 <b> Был запланирован , но не был СОВЕРШЕН!!!!!

In [359]:
df_был_план_нет_факта = df_емиас.merge(df_емиас[['Дата протокола','id_data_fact']].add_suffix('_факт'), how='left', left_on='id_data_plan', right_on='id_data_fact_факт')

In [360]:
# df_был_план_нет_факта[
#                 df_был_план_нет_факта['Дата протокола_факт'].isna()
#                 #  (df_был_план_нет_факта['Дата протокола'] == '2025-01-10')
#                 & (df_был_план_нет_факта['план_дата_визит_врач'] == '2025-01-10')
#                 # & (df_был_план_нет_факта['ОВПП'] == 'Филиал "Бутово" ММЦПП ДЗМ')
#                 & (df_был_план_нет_факта['ОВПП'] == 'Филиал "Коломенское" ММЦПП ДЗМ')
#                 # & (df_был_план_нет_факта['ID_пациента_str'] == '16000059')
#                 ]

In [361]:
# -------------------------------------------------------- откидываем ЕСЛИ не было факта в запланированную дату ----------------
df_был_план_нет_факта = df_был_план_нет_факта[
                df_был_план_нет_факта['Дата протокола_факт'].isna()
                # & (df_был_план_нет_факта['план_дата_визит_врач'] == '2025-01-10')
                # # & (df_был_план_нет_факта['ОВПП'] == 'Филиал "Бутово" ММЦПП ДЗМ')
                # & (df_был_план_нет_факта['ОВПП'] == 'Филиал "Коломенское" ММЦПП ДЗМ')
                ]

In [362]:
# **********************************************
df_был_план_нет_факта[
                # ~df_был_план_нет_факта['Дата протокола_факт'].isna()
                 (df_был_план_нет_факта['Дата протокола'] == '2025-01-10')
                #  (df_был_план_нет_факта['план_дата_визит_врач'] == '2025-01-10')
                # & (df_был_план_нет_факта['ОВПП'] == 'Филиал "Бутово" ММЦПП ДЗМ')
                & (df_был_план_нет_факта['ОВПП'] == 'Филиал "Коломенское" ММЦПП ДЗМ')
                # & (df_был_план_нет_факта['ID_пациента_str'] == '16000059')
                ]

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,id_data_fact,id_data_plan,Дата протокола_факт,id_data_fact_факт
24906,7700005136800750,28158773,2025-01-10,Дубовицкая А. В.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-08,28158773-2025-01-10,28158773-2025-02-08,NaT,NaN
24917,8176550829000103,30000007913368,2025-01-10,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-01-30,30000007913368-2025-01-10,30000007913368-2025-01-30,NaT,NaN
24924,7754040821000592,2242483780,2025-01-10,Дубовицкая А. В.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-07,2242483780-2025-01-10,2242483780-2025-02-07,NaT,NaN
24929,7700004116651152,1990357216,2025-01-10,Дубовицкая А. В.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-08,1990357216-2025-01-10,1990357216-2025-02-08,NaT,NaN
24933,7700003144220250,20946740,2025-01-10,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-01-29,20946740-2025-01-10,20946740-2025-01-29,NaT,NaN
24958,7700001115760451,579364766,2025-01-10,Беланова П. П.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-01-27,579364766-2025-01-10,579364766-2025-01-27,NaT,NaN
24970,7770060830000540,30268698,2025-01-10,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-01-29,30268698-2025-01-10,30268698-2025-01-29,NaT,NaN
24990,7700004054171059,17633439,2025-01-10,Дубовицкая А. В.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-01-29,17633439-2025-01-10,17633439-2025-01-29,NaT,NaN
24997,7700009060191156,16790023,2025-01-10,Дубовицкая А. В.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-08,16790023-2025-01-10,16790023-2025-02-08,NaT,NaN
25002,7769270888000416,28794796,2025-01-10,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-07,28794796-2025-01-10,28794796-2025-02-07,NaT,NaN


In [363]:
# ------------------------------------------------------------------ ['Вопрос для разбора' ---------------
df_был_план_нет_факта['дефект'] = "был_план_нет_факта"
df_был_план_нет_факта['Вопрос для разбора'] = "причина переноса визита на другой день"
# -------------- если нет даты визита  для этого пациента ----- 
# df_был_план_нет_факта['Вопрос для разбора'] = "причина, по которой пациент не включен в план наряд (нет в ЕМИАС)"

In [364]:
df_был_план_нет_факта = df_был_план_нет_факта[[
    'ОМС_пациента_str',
 'ID_пациента_str',
 'Дата протокола',
 'ФИО врача',
 'ОВПП',
 'план_дата_визит_врач',
 'id_data_plan',
 'id_data_fact',
 'Дата протокола_факт',
#  'id_data_fact_факт',
 'дефект',
 'Вопрос для разбора'

]]

In [365]:
# ***************
df_был_план_нет_факта[df_был_план_нет_факта['ID_пациента_str'] == '16000059']

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,id_data_plan,id_data_fact,Дата протокола_факт,дефект,Вопрос для разбора
25055,7774060891001086,16000059,2025-01-10,Глебова Л. Б.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2025-02-07,16000059-2025-02-07,16000059-2025-01-10,NaT,был_план_нет_факта,причина переноса визита на другой день
40003,7774060891001086,16000059,2024-09-05,Дубовицкая А. В.,"Филиал ""Коломенское"" ММЦПП ДЗМ",2024-10-05,16000059-2024-10-05,16000059-2024-09-05,NaT,был_план_нет_факта,причина переноса визита на другой день


In [366]:
df_был_план_нет_факта.shape

(22876, 11)

In [367]:
# df_был_план_нет_факта = df_был_план_нет_факта[
#                 df_был_план_нет_факта['Дата протокола_факт'].isna()
#                 # & (df_был_план_нет_факта['план_дата_визит_врач'] == '2025-01-10')
#                 # # & (df_был_план_нет_факта['ОВПП'] == 'Филиал "Бутово" ММЦПП ДЗМ')
#                 # & (df_был_план_нет_факта['ОВПП'] == 'Филиал "Коломенское" ММЦПП ДЗМ')
#                 ]
# df_был_план_нет_факта.shape

<div style="background-color:#B2EBF2; color:black;  font-size:10pt; border: 1px solid #35BDD0; border-radius: 5px; padding:10px" /div>
 <b> НЕ БЫЛО в ПЛАНЕ, но БЫЛ ФАКТ!!! (к факту цепляем план)

In [368]:
df_факт_по_плану = df_емиас.merge(df_емиас[['id_data_plan', 'Дата протокола']].add_suffix('_план'), how='left', left_on='id_data_fact', right_on='id_data_plan_план')

In [371]:
df_факт_по_плану.shape

(42625, 10)

<div style="background-color:green; color:black;  font-size:10pt; border: 1px solid #35BDD0; border-radius: 5px; padding:10px" /div>
 <b> визит_не_по_плану!!!!!
 df_визит_не_по_плану

In [370]:
df_визит_не_по_плану = df_факт_по_плану[df_факт_по_плану['id_data_plan_план'].isna()]
df_визит_не_по_плану

,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,id_data_fact,id_data_plan,id_data_plan_план,Дата протокола_план
0,7701004055610961,19648959,2024-09-30,Борисова Т. П.,"Филиал ""Царицыно"" ММЦПП ДЗМ",2024-10-14,19648959-2024-09-30,19648959-2024-10-14,NaN,NaT
1,7774160878000109,18131229,2024-09-30,Разгуляев А. В.,"Филиал ""Куркино"" ММЦПП ДЗМ",2024-10-17,18131229-2024-09-30,18131229-2024-10-17,NaN,NaT
3,7774360877000627,2242663271,2024-09-30,Ледяева О. В.,"Филиал ""Вороново"" ММЦПП ДЗМ",2024-10-31,2242663271-2024-09-30,2242663271-2024-10-31,NaN,NaT
4,7700006160270852,20152756,2024-09-30,Казак А. А.,"Филиал ""Даниловский"" ММЦПП ДЗМ",2024-10-05,20152756-2024-09-30,20152756-2024-10-05,NaN,NaT
5,7701004147080677,16684078,2024-09-30,Лямзин С. А.,"Филиал ""Люблино"" ОРП ММЦПП ДЗМ",2024-10-30,16684078-2024-09-30,16684078-2024-10-30,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
42601,7767560869000191,16484988,2025-01-12,Гладенкова А. О.,"Филиал ""Солнцево"" ММЦПП ДЗМ",2025-02-09,16484988-2025-01-12,16484988-2025-02-09,NaN,NaT
42612,4072260869000229,1029275191,2024-09-24,Шим Н. Н.,"Филиал ""Куркино"" ММЦПП ДЗМ",2024-10-24,1029275191-2024-09-24,1029275191-2024-10-24,NaN,NaT
42618,7700001164070648,20815226,2024-09-19,Егоров Е. В.,"Филиал ""Дегунино"" ММЦПП ДЗМ",2024-10-03,20815226-2024-09-19,20815226-2024-10-03,NaN,NaT
42623,7774460895000219,10158361,2024-09-08,Уварова О. В.,"Филиал ""Преображенское"" ММЦПП ДЗМ",2024-09-22,10158361-2024-09-08,10158361-2024-09-22,NaN,NaT


In [372]:
df_визит_не_по_плану['дефект'] = "визит_не_по_плану"
df_визит_не_по_плану['Вопрос для разбора'] = "причина, по которой пациент включен в план наряд"

In [373]:
df_визит_не_по_плану = df_визит_не_по_плану[[
 'ОМС_пациента_str',
 'ID_пациента_str',
 'Дата протокола',
 'ФИО врача',
 'ОВПП',
 'план_дата_визит_врач',
 'id_data_plan',
 'id_data_fact',
#  'Дата протокола_факт',
#  'id_data_fact_факт',
 'дефект',
 'Вопрос для разбора'


]]

In [374]:
df_дефект = pd.concat([df_был_план_нет_факта,df_визит_не_по_плану]).reset_index()

In [375]:
df_дефект_с_обл = df_дефект.merge(df_obl[['ФИО пациента','Дата рождения',
                         'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях',
                         'Текущий статус пациента',
                         'ID_пациента_str']].add_suffix('_обл'), how='left', left_on='ID_пациента_str', right_on='ID_пациента_str_обл')

In [ ]:
# ----------------------------------- Цепляем ДАТУ ближайшего визита !! -------------------



In [376]:
# df_ogig = pd.read_excel('z:/Проектный офис/Протоколы/Все/Протокол ЕМИАС все визиты_20250123_test.xlsx')

In [377]:
df = df_дефект_с_обл.copy()

In [378]:
df.columns.tolist()

['index',
 'ОМС_пациента_str',
 'ID_пациента_str',
 'Дата протокола',
 'ФИО врача',
 'ОВПП',
 'план_дата_визит_врач',
 'id_data_plan',
 'id_data_fact',
 'Дата протокола_факт',
 'дефект',
 'Вопрос для разбора',
 'ФИО пациента_обл',
 'Дата рождения_обл',
 'Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_обл',
 'Текущий статус пациента_обл',
 'ID_пациента_str_обл']

In [385]:
data_plan = pd.Timestamp('2025-01-10')

In [386]:
list_id_pac = df.loc[((df['план_дата_визит_врач'] == data_plan)
        # & (df['дефект'] == 'был_план_нет_факта')
        )]['ID_пациента_str'].unique().tolist()

In [387]:
# list_id_pac

In [397]:
# id_pac = 30488116
[df['Дата протокола'] - data_plan]

[0       -102 days
 1       -102 days
 2       -102 days
 3       -102 days
 4       -102 days
            ...   
 46028      2 days
 46029   -108 days
 46030   -113 days
 46031   -124 days
 46032   -134 days
 Name: Дата протокола, Length: 46033, dtype: timedelta64[ns]]

In [398]:
# datetime.timedelta(days=0)

In [399]:
for i in list_id_pac:
    df.loc[df['ID_пациента_str'] == i, 'кол-во_дней'] = df['Дата протокола'] - data_plan
    df.loc[df['ID_пациента_str'] == i, 'кол-во_дней_abs'] = df['кол-во_дней'].abs()
    # fakt_blijaishaia = df.loc[((df['ID пациента'] == i) & (df['кол-во_дней_abs'] == df['кол-во_дней_abs'].min()))].sort_values(by=['ID пациента', 'план_дата_визит_врач', 'кол-во_дней_abs'], ascending=[True, True, True])['Дата протокола'].values
    fakt_blijaishaia = df.loc[((df['ID_пациента_str'] == i))].sort_values(by=['ID_пациента_str', 'кол-во_дней_abs'], ascending=[True, True]).reset_index().sort_values(by=['ID_пациента_str', 'план_дата_визит_врач', 'кол-во_дней_abs'], ascending=[True, True, True]).loc[0, 'Дата протокола']
    df.loc[((df['ID_пациента_str'] == i) & (df['план_дата_визит_врач'] == data_plan) & (df['кол-во_дней_abs'] != '0 days')), 'Фактическая дата визита согласно ЕМИАС'] = fakt_blijaishaia
    # print(fakt_blijaishaia)
    # df.loc[((df['ID_пациента_str'] == i) & (df['план_дата_визит_врач'] == data_plan) & (df['кол-во_дней_abs'] != '0 days')), 'Фактическая дата визита согласно ЕМИАС'] = df.loc[((df['ID_пациента_str'] == i) & (df['кол-во_дней_abs'] == df['кол-во_дней_abs'].min()))]['Дата протокола'].values

    # fakt_blijaishaia = df_vse.loc[df_vse['кол-во_дней_abs'] == df_vse['кол-во_дней_abs'].min()]['Дата протокола'].values
    # df_vse.loc[((df_vse['ID_пациента_str'] == i) & (df_vse['план_дата_визит_врач'] == data_plan)), 'Фактическая дата визита согласно ЕМИАС'] = fakt_blijaishaia
    

In [406]:
df[~df['Фактическая дата визита согласно ЕМИАС'].isna()
   &( (df['ОВПП'] == 'Филиал "Бутово" ММЦПП ДЗМ'))
   ]

,index,ОМС_пациента_str,ID_пациента_str,Дата протокола,ФИО врача,ОВПП,план_дата_визит_врач,id_data_plan,id_data_fact,Дата протокола_факт,дефект,Вопрос для разбора,ФИО пациента_обл,Дата рождения_обл,"Медицинская организация, к которой прикреплен пациент для оказания паллиативной специализированной медицинской помощи в амбулаторных условиях_обл",Текущий статус пациента_обл,ID_пациента_str_обл,кол-во_дней,кол-во_дней_abs,Фактическая дата визита согласно ЕМИАС
6203,16687,3778550872000037,30000017189008,2024-12-09,Алексеенко А. Л.,"Филиал ""Бутово"" ММЦПП ДЗМ",2025-01-10,30000017189008-2025-01-10,30000017189008-2024-12-09,NaT,был_план_нет_факта,причина переноса визита на другой день,Доманова Нина Яковлевна,1944-01-27,Бутово ОВПП,Сопровождение приостановлено,30000017189008,-32 days,32 days,2024-12-09
6233,16765,7758220846003278,2239045817,2024-12-09,Алексеенко А. Л.,"Филиал ""Бутово"" ММЦПП ДЗМ",2025-01-10,2239045817-2025-01-10,2239045817-2024-12-09,NaT,был_план_нет_факта,причина переноса визита на другой день,Никулин Сергей Михайлович,1977-01-03,Бутово ОВПП,На сопровождении,2239045817,-32 days,32 days,2024-12-26
6283,16924,7772660872000205,17787791,2024-12-10,Абрамова Т. П.,"Филиал ""Бутово"" ММЦПП ДЗМ",2025-01-10,17787791-2025-01-10,17787791-2024-12-10,NaT,был_план_нет_факта,причина переноса визита на другой день,Подвербная Мария Алексеевна,1933-07-27,Бутово ОВПП,Оказание ПМП прекращено,17787791,-31 days,31 days,2024-12-10
6300,16982,7700000062090543,16725115,2024-12-10,Алексеенко А. Л.,"Филиал ""Бутово"" ММЦПП ДЗМ",2025-01-10,16725115-2025-01-10,16725115-2024-12-10,NaT,был_план_нет_факта,причина переноса визита на другой день,Леонтьева Надежда Ивановна,1943-05-09,Бутово ОВПП,Сопровождение приостановлено,16725115,-31 days,31 days,2024-12-10
6301,16983,7778940888000595,20301245,2024-12-10,Алексеенко А. Л.,"Филиал ""Бутово"" ММЦПП ДЗМ",2025-01-10,20301245-2025-01-10,20301245-2024-12-10,NaT,был_план_нет_факта,причина переноса визита на другой день,Матушкина Лариса Александровна,1950-01-11,Бутово ОВПП,Сопровождение приостановлено,20301245,-31 days,31 days,2024-12-10
6304,16988,7700004251251236,17547227,2024-12-10,Алексеенко А. Л.,"Филиал ""Бутово"" ММЦПП ДЗМ",2025-01-10,17547227-2025-01-10,17547227-2024-12-10,NaT,был_план_нет_факта,причина переноса визита на другой день,Гришина Нэлина Ивановна,1936-12-25,Бутово ОВПП,Оказание ПМП прекращено,17547227,-31 days,31 days,2024-12-10
6310,17001,7777460884001506,23696974,2024-12-10,Алексеенко А. Л.,"Филиал ""Бутово"" ММЦПП ДЗМ",2025-01-10,23696974-2025-01-10,23696974-2024-12-10,NaT,был_план_нет_факта,причина переноса визита на другой день,Афонина Валентина Серафимовна,1935-02-15,Бутово ОВПП,Сопровождение приостановлено,23696974,-31 days,31 days,2024-12-10
7516,20587,6278150843000048,1009116516,2024-12-24,Баранов Д. Д.,"Филиал ""Бутово"" ММЦПП ДЗМ",2025-01-10,1009116516-2025-01-10,1009116516-2024-12-24,NaT,был_план_нет_факта,причина переноса визита на другой день,Кожевин Геннадий Анатольевич,1948-01-06,Бутово ОВПП,На сопровождении,1009116516,-17 days,17 days,2025-01-11
7579,20705,7771650840000339,531979016,2024-12-24,Баранов Д. Д.,"Филиал ""Бутово"" ММЦПП ДЗМ",2025-01-10,531979016-2025-01-10,531979016-2024-12-24,NaT,был_план_нет_факта,причина переноса визита на другой день,Иванов Михаил Павлович,1943-08-09,Бутово ОВПП,На сопровождении,531979016,-17 days,17 days,2025-01-12
9052,23149,7700003014531056,16909909,2025-01-03,Абрамова Т. П.,"Филиал ""Бутово"" ММЦПП ДЗМ",2025-01-10,16909909-2025-01-10,16909909-2025-01-03,NaT,был_план_нет_факта,причина переноса визита на другой день,Михайлов Виктор Витальевич,1956-10-03,Бутово ОВПП,Сопровождение приостановлено,16909909,-7 days,7 days,2025-01-03


In [ ]:
# df.loc[((df['ID пациента'] == i) & (df['кол-во_дней_abs'] == df['кол-во_дней_abs'].min()))].sort_values(by=['ID пациента', 'план_дата_визит_врач', 'кол-во_дней_abs'], ascending=[True, True, True])['Дата протокола'].values

In [ ]:
# df.loc[((df['ID пациента'] == i) & (df['план_дата_визит_врач'] == data_plan) & (df['кол-во_дней_abs'] != '0 days'))].sort_values(by=['ID пациента', 'план_дата_визит_врач', 'кол-во_дней_abs'], ascending=[True, True, True]).reset_index().sort_values(by=['ID пациента', 'план_дата_визит_врач', 'кол-во_дней_abs'], ascending=[True, True, True]).loc[0, 'Дата протокола'].values

In [ ]:
# df.loc[((df['ID_пациента_str'] == i) & (df['план_дата_визит_врач'] == data_plan) & (df['кол-во_дней_abs'] != '0 days'))]['Дата протокола']

In [ ]:
# df.loc[((df['ID_пациента_str'] == i) & (df['план_дата_визит_врач'] == data_plan) & (df['кол-во_дней_abs'] != '0 days'))].drop_duplicates()

In [ ]:
df_save = df.loc[df['план_дата_визит_врач'] == data_plan]

In [ ]:
df_save.to_excel('c:/Users/1/Desktop/выгрузки/plan_fakt.xlsx')

In [171]:
df_1001_пф_new_Б = df_дефект_с_обл.loc[
                # df_дефект_с_обл['Дата протокола'].isna()
              (  ( (df_дефект_с_обл['дефект'] == 'был_план_нет_факта') & (df_дефект_с_обл['план_дата_визит_врач'] == '2025-01-10'))  
                | ((df_дефект_с_обл['дефект'] == 'визит_не_по_плану') & (df_дефект_с_обл['Дата протокола'] == '2025-01-10'))
              ) 
                & (df_дефект_с_обл['ОВПП'] == 'Филиал "Бутово" ММЦПП ДЗМ')
                ]

In [ ]:
df_1001_пф_new_Б

In [164]:
with pd.ExcelWriter('df_1001_пф_new' + '.xlsx',
                      engine="xlsxwriter",
                      date_format='DD.MM.YYYY',
                      datetime_format='DD.MM.YYYY') as writer:
    df_1001_пф_new.to_excel(writer, freeze_panes=(2,0), index = False)

In [ ]:
df_1001_пф_new

In [167]:
df_1001_пф_new = df_дефект_с_обл.loc[
                # df_дефект_с_обл['Дата протокола'].isna()
              (  ( (df_дефект_с_обл['дефект'] == 'был_план_нет_факта') & (df_дефект_с_обл['план_дата_визит_врач'] == '2025-01-10'))  
                | ((df_дефект_с_обл['дефект'] == 'визит_не_по_плану') & (df_дефект_с_обл['Дата протокола'] == '2025-01-10'))
              ) 
                # & (df_дефект_с_обл['ОВПП'] == 'Филиал "Бутово" ММЦПП ДЗМ')
                ]

In [ ]:
df_1001_пф_new.shape

In [ ]:
with pd.ExcelWriter('df_1001_пф_new' + '.xlsx',
                      engine="xlsxwriter",
                      date_format='DD.MM.YYYY',
                      datetime_format='DD.MM.YYYY') as writer:
    df_1001_пф_new.to_excel(writer, freeze_panes=(2,0), index = False)

In [ ]:
df_дефект_с_обл